In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
import os
sys.path.append(os.path.abspath('../utils/'))

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
import json
from dwh_utils import *
from transformation_utils import *

In [0]:
with open("/Volumes/acme_ecommerce/source/data-sources/schema/customers.json", "r") as f:
    customers_schema = StructType.fromJson(json.load(f))

In [0]:
df = (
    spark.read.option("headerRows", 1).schema(customers_schema)
    .excel("/Volumes/acme_ecommerce/source/data-sources/customers/*.xlsx")
).withColumn("ingest_timestamp", current_timestamp()).select("*", "_metadata.file_path")

In [0]:
df = normalize_column_names(spark=spark, df=df)

In [0]:
write_to_delta_with_cdc_by_name(
    spark=spark, df=df, catalog="acme_ecommerce", schema="bronze", table="customers", merge_keys=["Customer ID"], append_only=True
)